In [4]:
import os
import sys

sys.path.insert(0, os.path.abspath('/home/max/uva/mds_python'))

from mds.models import *
from mds.models.compact.user import UserCompactView
from mds.database.mongo import GetConfig
from mds.database.minio import GetMinioConfig

In [11]:
from mds.models.user import User
from mds.models.dataset import Dataset
from mds.models.download import Download

In [21]:
mongo_client = GetConfig()
mongo_database = mongo_client["test"]
mongo_collection = mongo_database["test_col"]

In [31]:
minio_client = GetMinioConfig()

In [12]:
test_user = User(**{
		"@id": "ark:99999/test-user1",
		"name": "Test User1",
		"type": "Person",
		"email": "testuser1@example.org",
		"password": "test1",
		"organizations": [],
		"projects": [],
		"datasets": [],
		"software": [],
		"computations": [],
		"evidencegraphs": []
		})

In [22]:
create_user_status = test_user.create(mongo_collection)

In [24]:
create_user_status.success

True

In [43]:
test_dataset = Dataset(**{
	"@id": "ark:99999/test-dataset",
	"@type": "evi:Dataset",
	"name": "test dataset",
	"owner": {
		"@id": "ark:99999/test-user1",
		"@type": "Person",
		"name": "Test User1",
		"email": "testuser1@example.org"
		}
    })

In [44]:
create_dataset = test_dataset.create(mongo_collection)

In [45]:
create_dataset.success

True

In [49]:
test_dataset.id

'ark:99999/test-dataset'

In [46]:
test_data_download = Download(**{
	"@id": "ark:99999/test-download",
	"name": "test file.txt",
	"@type": "DataDownload",
	"encodingFormat": ".txt",
	"encodesCreativeWork": test_dataset.id,
    })

In [47]:
create_data_download = test_data_download.create_metadata(mongo_client)

In [48]:
str(create_data_download)

'Success: False\tMessage: dataset ark:99999/test-dataset not found\tStatusCode: 404'

In [32]:
# upload data file
test_file = open("tests/test-data.csv")

upload_file = test_data_download.register(test_file, mongo_client, minio_client)


In [37]:
test_file.close()
str(upload_file)

'Success: False\tMessage: dataDownload ark:99999/test-download not found\tStatusCode: 404'

In [ ]:
test_user = UserCompactView(
    id="ark:99999/test-owner",
    name="test owner1",
    email="testowner1@example.org"
)

test_software = Software(
    id="ark:99999/test-software",
    owner=owner_inst1,
    citation="doi://blabla"
)

test_software_download = DataDownload()

test_dataset = Dataset()

test_dataset_download = DataDownload()

test_computation  =  Computation(
        id="ark:99999/test-comp",
        name="",
        owner=self.test_user,
        author="author1",
        dateCreated="2022-02-15",
        dateFinished="2022-02-15",
        usedSoftware=[software_inst1]
    )

In [ ]:
test_computation.run_custom_container()

In [ ]:
register_computation(test_computation.containerId)